<a href="https://colab.research.google.com/github/TurkuNLP/Deep_Learning_in_LangTech_course/blob/master/hf_seq2seq_lemmatization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The task

* Lemmatization
* Input: wordform + morpho information
* Output: word baseform
* Easy for English, but not so much for Finnish or many other languages

Here is few examples:

* dogs+NOUN+Plural -> dog
* sheep+NOUN+Plural -> sheep
* voi+VERB+... -> voida
* voi+NOUN+Singular -> voi

# Data preparation

* We can use universaldependencies.org
* Collection of treebanks
* Pick your favorite language, I will use Finnish

In [1]:
!pip3 install --quiet datasets transformers accelerate evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.1 MB/s eta 0:00:00


You can use e.g. UD_English-EWT for English or any other language you want from UniversalDependencies

In [2]:
!wget -q -O train.conllu https://github.com/UniversalDependencies/UD_Finnish-TDT/raw/master/fi_tdt-ud-train.conllu
!wget -q -O validation.conllu https://github.com/UniversalDependencies/UD_Finnish-TDT/raw/master/fi_tdt-ud-dev.conllu
!wget -q -O test.conllu https://github.com/UniversalDependencies/UD_Finnish-TDT/raw/master/fi_tdt-ud-test.conllu

# Data preparation

* The CoNLL format should be familiar to you by now
* Here is few lines (the delimiter is TAB)



```
# newdoc id = weblog-blogspot.com_zentelligence_20040423000200_ENG_20040423_000200
# sent_id = weblog-blogspot.com_zentelligence_20040423000200_ENG_20040423_000200-0001
# newpar id = weblog-blogspot.com_zentelligence_20040423000200_ENG_20040423_000200-p0001
# text = What if Google Morphed Into GoogleOS?
1	What	what	PRON	WP	PronType=Int	0	root	0:root	_
2	if	if	SCONJ	IN	_	4	mark	4:mark	_
3	Google	Google	PROPN	NNP	Number=Sing	4	nsubj	4:nsubj	_
4	Morphed	morph	VERB	VBD	Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin	1	advcl	1:advcl:if	_
5	Into	into	ADP	IN	_	6	case	6:case	_
6	GoogleOS	GoogleOS	PROPN	NNP	Number=Sing	4	obl	4:obl:into	SpaceAfter=No
7	?	?	PUNCT	.	_	4	punct	4:punct	_


```

* Let us form training examples like so:
    * Input is `wordform`_`POS`_`FEATS`
    * Output is the lemma
* We can reuse part of our dataset preparation code from the [MLP notebook](https://github.com/TurkuNLP/Deep_Learning_in_LangTech_course/blob/master/hf_trainer_mlp.ipynb)


In [3]:
import json

In [4]:
ID,FORM,LEMMA,UPOS,XPOS,FEAT,HEAD,DEPREL,DEPS,MISC=range(10)

We now want to formulate the examples such that input is the word and all morphological information, output is the lemma

```
IN: Morphed+++VERB|Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin

OUT: morph
```

In [5]:
def yield_examples(fname,uniq=True):
    """
    uniq: do unique on the words, not to have duplicated examples for punctuation and stuff
    """
    with open(fname) as f:
        seen=set()
        for line in f:
            line=line.rstrip("\n")
            if not line or line.startswith("#"): #empty and comment lines: skip
                continue
            cols=line.split("\t")
            if not cols[0].isnumeric(): #lines which are not a real word: skip
                continue
            #form the example pair:
            #   IN: wordform+++POSTAG|all other tags
            #  OUT: lemma
            form_tags,lemma=cols[FORM]+"+++"+cols[UPOS]+"|"+cols[FEAT],cols[LEMMA]
            if uniq:
                if (form_tags,lemma) in seen:
                    continue
                seen.add((form_tags,lemma))
            #and here is the example
            yield {"form_tags":form_tags,"lemma":lemma}

* turn every `.conllu` into the corresponding `.jsonl` with the examples
* that way we can then easily load it as a dataset and train a model



In [6]:
for fname in ("train.conllu","validation.conllu","test.conllu"):
    with open(fname.replace(".conllu",".jsonl"),"wt") as f_out:
        for example in yield_examples(fname):
            print(json.dumps(example,ensure_ascii=False,sort_keys=True),file=f_out)

## Load as dataset

* This is a slight modification of the loading code we've been using throughout the course


In [7]:
import datasets
dataset = datasets.load_dataset(
    'json',                             # Format of the data
    data_files={"train":"train.jsonl","validation":"validation.jsonl","test":"test.jsonl"},
    split={
        "train":"train",
        "validation":"validation",
        "test":"test"
    },
    features=datasets.Features({    # Here we tell how to interpret the attributes
        "form_tags":datasets.Value("string"),
        "lemma":datasets.Value("string")
    })
)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [8]:
# this is always a good idea as we have learned!
dataset=dataset.shuffle()

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['form_tags', 'lemma'],
        num_rows: 51096
    })
    validation: Dataset({
        features: ['form_tags', 'lemma'],
        num_rows: 8660
    })
    test: Dataset({
        features: ['form_tags', 'lemma'],
        num_rows: 9400
    })
})

# Tokenize and prepare

* This is a bit more cpmplex than it might sound
* Let's stop to think; do we really want to tokenize this data in the usual manner?

the examples are formed surprisingly similarly to what you've seen before:

* `input_ids` is the input side
* `attention_mask` is the input attention mask
* `labels` is the output ids
* the encoder-decoder model should (and hopefully does) take care of the rest
* it is a good idea to mark sequence start and end for the model both on the input and the output side
* we can tell the tokenizer to use `[unused1]` and `[unused2]` as the beginning/end of sequence tokens


In [10]:
import transformers

#OK, let's try with our trusty tokenizer
#but why would this work in the first place?
model_name = "TurkuNLP/bert-base-finnish-cased-v1"

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name,bos_token="[unused1]",eos_token="[unused2]")

def tokenize(example):
    # let's get the input word separated from the tags
    inp_w,inp_tags=example["form_tags"].split("+++",1)
    out=" ".join(example["lemma"])

    # make sure you separate everything by space, the tokenizer will pick it up
    # below I print one of the input/output pairs so check that out
    inp_tok=tokenizer("[unused1]"+" "+" ".join(inp_w)+" "+(inp_tags.replace("|"," "))+" "+"[unused2]",truncation=True)
    outp_tok=tokenizer("[unused1]"+" "+out+" "+"[unused2]",truncation=True)
    return {"input_ids":inp_tok["input_ids"],
            "attention_mask":inp_tok["attention_mask"],
            "labels":outp_tok["input_ids"]}

In [11]:
dataset=dataset.map(tokenize)

Map:   0%|          | 0/51096 [00:00<?, ? examples/s]

Map:   0%|          | 0/8660 [00:00<?, ? examples/s]

Map:   0%|          | 0/9400 [00:00<?, ? examples/s]

In [12]:
print(" IN:",tokenizer.convert_ids_to_tokens(dataset["train"][0]["input_ids"]))
print("OUT:",tokenizer.convert_ids_to_tokens(dataset["train"][0]["labels"]))

 IN: ['[CLS]', '[unused1]', 'm', 'ä', 'e', 'n', 'l', 'a', 's', 'k', 'u', 'k', 'i', 'l', 'p', 'a', 'i', 'l', 'u', 'n', 'NO', '##UN', 'Cas', '##e', '=', 'Gen', 'Der', '##ivat', '##ion', '=', 'U', 'Nu', '##mb', '##er', '=', 'Sing', '[unused2]', '[SEP]']
OUT: ['[CLS]', '[unused1]', 'm', 'ä', 'e', 'n', '[UNK]', 'l', 'a', 's', 'k', 'u', '[UNK]', 'k', 'i', 'l', 'p', 'a', 'i', 'l', 'u', '[unused2]', '[SEP]']


# Encoder - Decoder model

* We shall use a "vanilla" encoder-decoder model
* Luckily, it is still relatively easy
* Let us train a small model 128-long embeddings, 4 layers, 4 attention heads

In [13]:
config_encoder = transformers.BertConfig(vocab_size=tokenizer.vocab_size,
                                         hidden_size=128,
                                         num_hidden_layers=4,
                                         num_attention_heads=4,
                                         )
config_decoder = transformers.BertConfig(vocab_size=tokenizer.vocab_size,
                                         hidden_size=128,
                                         num_hidden_layers=4,
                                         num_attention_heads=4,
                                         decoder_start_token_id=53)
config = transformers.EncoderDecoderConfig.from_encoder_decoder_configs(config_encoder, config_decoder)
config.decoder_start_token_id=53 #avoids an Error
config.pad_token_id=0            #avoids an Error
model = transformers.EncoderDecoderModel(config=config)

This is useful to run:

`help(model.forward)`

There is a Seq2Seq collator

In [14]:
#help(model.forward)

In [15]:
collator=transformers.DataCollatorForSeq2Seq(tokenizer=tokenizer,
                                             model=model,
                                             padding=True,
                                             return_tensors="pt")

In [16]:
trainer_args = transformers.Seq2SeqTrainingArguments(
    "checkpoints",
    evaluation_strategy="steps",
    logging_strategy="steps",
    load_best_model_at_end=True,
    eval_steps=1000,
    logging_steps=100,
    learning_rate=1e-4,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    max_steps=15000,
    save_steps=1000,
    predict_with_generate=True #this did take a while to figure out !
)

In [17]:
early_stopping = transformers.EarlyStoppingCallback(
    early_stopping_patience=3
)

In [18]:
trainer = transformers.Seq2SeqTrainer(
    model=model,
    args=trainer_args,
    data_collator=collator,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer = tokenizer,
    callbacks=[early_stopping]
)

In [19]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Step,Training Loss,Validation Loss
1000,0.753000,0.590846
2000,0.344000,0.265871
3000,0.244400,0.188214
4000,0.197400,0.138092
5000,0.157800,0.115099
6000,0.140200,0.101394
7000,0.125400,0.089634
8000,0.115800,0.081286
9000,0.105600,0.075370
10000,0.102900,0.069570


/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no 

TrainOutput(global_step=15000, training_loss=0.34043788471221925, metrics={'train_runtime': 1587.5787, 'train_samples_per_second': 604.694, 'train_steps_per_second': 9.448, 'total_flos': 2184564452224128.0, 'train_loss': 0.34043788471221925, 'epoch': 18.77})

In [20]:
trainer.model.save_pretrained("s2s_lemmatizer")

In [21]:
dataset=dataset.shuffle()
test_data=dataset["test"].select(range(33))
predictions=trainer.predict(test_data)
for x,e in zip(predictions.predictions,test_data):
    print("------------------")
    print(">> ",e["form_tags"])
    print(tokenizer.decode(x))
    print()

/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


------------------
>>  mukava+++ADJ|Case=Nom|Degree=Pos|Number=Sing
[unused52] [CLS] [unused1] m u k a v a [unused2] [SEP] [unused2] [SEP] [unused2] [SEP] m a [unused2] [SEP] [unused1]

------------------
>>  2010+++NUM|NumType=Card
[unused52] [CLS] [unused1] 2 0 1 0 [unused2] [SEP] [unused2] [SEP] [unused2] [SEP] [unused2] [SEP] 2 [unused2] [SEP] [unused1] 2

------------------
>>  toimittava+++VERB|Case=Nom|Number=Sing|PartForm=Pres|VerbForm=Part|Voice=Pass
[unused52] [CLS] [unused1] t o i m i a [unused2] [SEP] [unused1] t a [unused2] [SEP] [unused1] t [unused2] [SEP]

------------------
>>  tilanteisiin+++NOUN|Case=Ill|Number=Plur
[unused52] [CLS] [unused1] t i l a n t e i n e n [unused2] [SEP] [unused1] t [unused2] [SEP]

------------------
>>  jokaiseksi+++PRON|Case=Tra|Number=Sing|PronType=Ind
[unused52] [CLS] [unused1] j o k a i n e n [unused2] [SEP] [unused1] j a [unused2] [SEP] [unused1] v

------------------
>>  mittaa+++VERB|Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=